In [1]:
!pip install pyspark
import pyspark
import os

from pyspark.sql.functions import col
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
from pyspark.sql.types import StructType, StructField, StringType, DateType

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=d1de5e6c0329d196646f5ae4683a8dd74bf07f769052487d4739758b2e83c85e
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [2]:
sc = SparkSession.builder.appName("Lab2").config("spark.jars.packages", "com.databricks:spark-xml_2.12:0.13.0").getOrCreate()

In [3]:
languages = sc.read.csv("programming-languages.csv")
languages = [str(x[0]) for x in languages.collect()]

In [6]:
posts = sc.read.format('xml').options(rowTag='row').load("posts_sample.xml")

In [7]:
from datetime import datetime
def get_year(x, year):
  return x._CreationDate >= datetime(year=year, month=1, day=1) and x._CreationDate <= datetime(year=year, month=12, day=31)

In [8]:
def text_lower(x):
  tag = next((language for language in languages if "<" + language.lower()+ ">" in x._Tags.lower()), None)
  return (x._Id, tag) if tag is not None else None

In [11]:
dataframe = {}
def create_df(year):
  # провека наличия тэга и корректности года (2010-2020)
  # нижний регистр, если находим язык, создаем кортеж, убираем пустые значения
  # группировка по значению в кортеже и агреггирование
  # сортировка по убыванию и создание dataframe
  dataframe[year] = posts.rdd.filter(lambda x: x._Tags is not None and get_year(x, year))\
      .map(text_lower).filter(lambda x: x is not None)\
      .keyBy(lambda x: x[1]).aggregateByKey(0, lambda x, y: x + 1, lambda x1, x2: x1 + x2)\
      .sortBy(lambda x: x[1], ascending=False).toDF()
  dataframe[year] = dataframe[year].select(col("_1").alias("Lang"),col("_2").alias(f"Count_in_{year}")).limit(10)
  dataframe[year].show()

In [14]:
for y in range(2010, 2020):
  create_df(y)

+-----------+-------------+
|       Lang|Count_in_2010|
+-----------+-------------+
|       Java|           52|
| JavaScript|           44|
|        PHP|           42|
|     Python|           25|
|Objective-C|           22|
|          C|           20|
|       Ruby|           11|
|     Delphi|            7|
|          R|            3|
|       Bash|            3|
+-----------+-------------+

+-----------+-------------+
|       Lang|Count_in_2011|
+-----------+-------------+
|        PHP|           97|
|       Java|           92|
| JavaScript|           82|
|     Python|           35|
|Objective-C|           33|
|          C|           24|
|       Ruby|           17|
|     Delphi|            8|
|       Perl|            8|
|       Bash|            7|
+-----------+-------------+

+-----------+-------------+
|       Lang|Count_in_2012|
+-----------+-------------+
|        PHP|          136|
| JavaScript|          129|
|       Java|          124|
|     Python|           65|
|Objective-C|     